In [1]:
from project2 import *

In [3]:
import torch
import numpy as np
import string
from collections import deque
from bert_serving.client import BertClient
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.data import Sentence
import flair

stacked_embeddings = WordEmbeddings('glove')

In [4]:
def get_stacked_embeddings(sentence):
    bert_embedding.embed(sentence)

def tensorToNumpy(t):
    return np.array(t[0].embedding.tolist())

def get_embeds(sentences):
    for i in sentences:
        bert_embedding.embed(i)
        
def strip_punct_tensor(tensor):
    return tensor.text.translate(str.maketrans('', '', string.punctuation))

def strip_punct(instr):
    return instr.translate(str.maketrans('', '', string.punctuation))

In [6]:
#bert_as_a_service = '172.17.0.19'
#bc = BertClient(ip=bert_as_a_service)

In [7]:
positive_vectors = [Sentence(i) for i in positive]
for i in positive_vectors:
    stacked_embeddings.embed(positive_vectors)
negative_vectors = [Sentence(i) for i in negative]
for i in positive_vectors:
    stacked_embeddings.embed(negative_vectors)
    

In [8]:
X_train_vec = np.array([i[0].embedding.cpu().numpy() for i in positive_vectors + negative_vectors])

In [9]:
vec = bc.encode([[i] for i in positive],is_tokenized=True,show_tokens=True)
positive_vectors = np.array([vec[0][i][0] for i in range(len(positive))])
vec = bc.encode([[i] for i in negative],is_tokenized=True,show_tokens=True)
negative_vectors = np.array([vec[0][i][0] for i in range(len(negative))])

NameError: name 'bc' is not defined

In [10]:
clf = LinearSVC(random_state=0, tol=1e-5)

In [11]:
X = X_train_vec
Y = np.concatenate( ( np.ones(len(positive_vectors)), np.ones(len(negative_vectors))*-1 ) )

In [14]:
clf.fit(X, Y)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=1e-05,
          verbose=0)

In [15]:
accuracy_score(clf.predict(X_train_vec), Y)

0.8999557978488286

# Get polarity scores for each of the documents

In [16]:
w = clf.coef_[0]
def polarity(w, document_vector):
    return np.dot(document_vector, w)

In [17]:
with open("test",'w') as f:
    f.write(str(w.tolist()))

In [18]:
pos_train_path = '../data401/reviews/train/pos/'
neg_train_path = '../data401/reviews/train/neg/'

pos_test_path = '../data401/reviews/test/pos/'
neg_test_path = '../data401/reviews/test/neg/'

In [19]:
words_of_interest = set(positive + negative)

In [34]:
def get_polarity(document, filepath, words_of_interest, w):
    
    sentences = sent_tokenize(document)
    
    sentences_vec = [Sentence(i, use_tokenizer=True) for i in sentences]
    
    for sentence in sentences_vec:
        stacked_embeddings.embed(sentence)
    
    words = []
    for s_idx, sentence in enumerate(sentences_vec):
        for w_idx, word in enumerate(sentence):
            if word.text in words_of_interest:
                words.append(sentences_vec[s_idx][w_idx].embedding)
                
    if (len(words) == 0):
        return np.nan
    else:
        return sum(words).cpu().numpy().dot(w)

In [35]:
with open(pos_train_path+pos_train_files[0]) as f:
    text = f.read().replace('<br />','\n')
    

In [22]:
get_polarity(text, "123", words_of_interest, w)

2.1661118944414492

In [199]:
import flair, torch

flair.device = torch.device('cuda:0')


In [36]:
rslts = []
import time
start = time.time()
measurement = 0
for i in pos_train_files:
    with open(pos_train_path+i) as f:
        measurement+= 1
        text = f.read().replace('<br />','\n')
        rslts.append((i, get_polarity(text, pos_train_path+i, words_of_interest, w)))
        if (measurement % 1000 == 0):
            end = time.time()
            print(end - start)
            start = end
            print(measurement)

12.71730637550354
1000
12.369462490081787
2000
13.241343259811401
3000
12.69118356704712
4000
12.801411867141724
5000
12.942528009414673
6000
12.6019606590271
7000
12.337921380996704
8000
12.999292373657227
9000
13.056675672531128
10000
12.951705694198608
11000
12.668541431427002
12000


In [32]:
tmp = pd.DataFrame(rslts)

In [37]:
tmp.to_csv("pos_train.csv")

In [188]:
files = []
for i in pos_train_files:
    with open(pos_train_path+i) as f:
        text = f.read().replace('<br />','\n')
        files.append((pos_train_path+i, text))

In [43]:
def get_polarity(sentence_vectors, tokens, filepath, w):
    
    rslt = []        
    for s_idx, sentence in enumerate(tokens):
        for w_idx, word in enumerate(sentence):
            if word in words_of_interest:
                rslt.append(sentence_vectors[s_idx][w_idx])
                
    if (len(rslt) == 0):
        pol = np.nan
    else:
        pol = sum(rslt).dot(w)   
        
    return ({'filepath':filepath, 'polarity': pol})
    
    
def get_polarities(base_path, filenames, w):
    
    rslts = []
    iterations = 0
    
    sentences = []

    sentence_mapping = []

    print("Building sentences")
    for filename in filenames:
    
        with open(base_path+filename) as f:
        
            text = f.read().replace('<br />','\n')
            input_sentence = sent_tokenize(text)
        
            start = len(sentences)
            end = start + len(input_sentence)
        
            sentences = sentences + input_sentence
            sentence_mapping.append((start, end))
    
    print("sentences built")
        
    index = 0
    current_batch_start = 0
    current_batch_end = 0
    queue = []
    for i,(j,k) in enumerate(sentence_mapping):
        if (current_batch_end - current_batch_start < 37000):
            current_batch_end = k
            queue.append((i,(j,k)))
        else:
            print("sending ~37k batched: Current at " + str(i) +" files...")
            (sentence_vectors, tokens) = bc.encode(sentences[current_batch_start:current_batch_end], show_tokens=True)
            for (ni,(ji,ki)) in queue:
                jj = ji - current_batch_start
                kj = ki - current_batch_start
                rslts.append(get_polarity(sentence_vectors[jj:kj], tokens[jj:kj], base_path+filenames[ni], w))
            queue = []
            current_batch_start = current_batch_end
            
    if (len(queue) > 0):
        print("cleaning up")
        (sentence_vectors, tokens) = bc.encode(sentences[current_batch_start:], show_tokens=True)
        for (ni,(ji,ki)) in queue:
            jj = ji - current_batch_start
            kj = ki - current_batch_start
            rslts.append(get_polarity(sentence_vectors[jj:kj], tokens[jj:kj], base_path+filenames[ni], w))
        queue = []
        current_batch_start = current_batch_end
        
    return rslts

In [44]:
print("making positive_train")
pos_train = get_polarities(pos_train_path, pos_train_files, w)
pd.DataFrame(pos_train).to_csv("pos_train.csv")

making positive_train
Building sentences
sentences built
sending ~37k batched: Current at 3045 files...
sending ~37k batched: Current at 6081 files...
sending ~37k batched: Current at 9065 files...
sending ~37k batched: Current at 12171 files...
cleaning up


In [45]:
print("making negative_train")

neg_train = get_polarities(neg_train_path, neg_train_files, w)
pd.DataFrame(neg_train).to_csv("neg_train.csv")

making negative_train
Building sentences
sentences built
sending ~37k batched: Current at 2906 files...
sending ~37k batched: Current at 5828 files...
sending ~37k batched: Current at 8707 files...
sending ~37k batched: Current at 11629 files...
cleaning up


In [46]:
print("making positive_test")

pos_test =  get_polarities(pos_test_path,  pos_test_files, w)
pd.DataFrame(pos_test).to_csv("pos_test.csv")

making positive_test
Building sentences
sentences built
sending ~37k batched: Current at 3108 files...
sending ~37k batched: Current at 6153 files...
sending ~37k batched: Current at 9302 files...
sending ~37k batched: Current at 12486 files...
cleaning up


In [47]:
print("making negative_test")

neg_test =  get_polarities(neg_test_path,  neg_test_files, w)
pd.DataFrame(neg_test).to_csv("neg_test.csv")

making negative_test
Building sentences
sentences built
sending ~37k batched: Current at 2952 files...
sending ~37k batched: Current at 5898 files...
sending ~37k batched: Current at 8834 files...
sending ~37k batched: Current at 11792 files...
cleaning up


In [ ]:
with open('../data401/reviews/imdb.vocab') as f:
    vocab = pd.Series(f.readlines())[:10000].apply(lambda x: x.rstrip())